In [106]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('titanic.csv')
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.isnull().sum()/df.shape[0]*100

In [ ]:
#check duplicates
df.duplicated().sum() 

In [ ]:
# Löschen der Spalte 'Cabin' aus dem DataFrame weil 77% missing values
df.drop('Cabin', axis=1, inplace=True)
#Age abrunden
df['Age'] = np.floor(df['Age'])
#missing values treatment
df['Age'].fillna(df['Age'].median(), inplace=True) # ist's sinvoll?
df['Age'] = df['Age'].astype(int)
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])


In [ ]:
#check missing values
df.isnull().sum()/df.shape[0]*100

In [115]:
#extrahiere new column 'Titel',das könnte wichtig für datenanalyse sein?
df["Title"] = df["Name"].apply(lambda x: x.split(",")[1].split(".")[0].strip())
df["Name"] = df["Name"].apply(lambda x: ' '.join([word for word in x.split() if not word.endswith('.')]))


In [ ]:
df.describe().T

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df

In [ ]:
# Import libraries
import base64
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, make_scorer, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, dcc, html

# 1. DATA PREPARATION
df = pd.read_csv('titanic.csv')

# Clean the data
# These columns (PassengerId, Name, Ticket, Cabin) are not useful for predicting survival. They either contain unique identifiers or non-informative text data that do not contribute to the model's predictive power.
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], errors='ignore')
# The 'Age' column has missing values. Using the median to fill these gaps is a common practice because it is robust to outliers and provides a central tendency measure.
df['Age'] = np.floor(df['Age'])
df['Age'].fillna(df['Age'].median(), inplace=True)
# The 'Embarked' column has missing values. Filling them with the most frequent value.
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
df = pd.get_dummies(df, columns=['Sex', 'Embarked'], drop_first=True)
# Features and target: To prepare the data for modeling, we need to separate the features (input variables) from the target (output variable).
X = df.drop(columns=['Survived'])
y = df['Survived']

# Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "KNN (k=3)": KNeighborsClassifier(n_neighbors=3)
}

# Cross-validation and Bootstrap methods
def cross_validate_method(model, X, y):
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    scoring = {
        'accuracy': 'accuracy',
        'precision': make_scorer(precision_score, average='weighted'),
        'recall': make_scorer(recall_score, average='weighted'),
        'f1': make_scorer(f1_score, average='weighted')
    }
    scores = cross_validate(model, X, y, cv=cv, scoring=scoring, return_estimator=True)
    
    confusion_matrices = []
    for estimator, (train_index, test_index) in zip(scores['estimator'], cv.split(X, y)):
        y_pred = estimator.predict(X[test_index])
        cm = confusion_matrix(y[test_index], y_pred)
        confusion_matrices.append(cm)
    
    mean_confusion_matrix = np.mean(confusion_matrices, axis=0)
    
    return {
        'accuracy': scores['test_accuracy'].mean(),
        'precision': scores['test_precision'].mean(),
        'recall': scores['test_recall'].mean(),
        'f1': scores['test_f1'].mean(),
        'confusion_matrix': mean_confusion_matrix
    }

def bootstrap_632_method(model, X, y):
    """Perform Bootstrap .632 method with extended metrics."""
    train_metrics, test_metrics = [], []
    confusion_matrices = []

    for _ in range(100):  # Number of bootstrap iterations
        # Bootstrap sampling
        bootstrap_indices = np.random.choice(np.arange(len(X)), size=len(X), replace=True)
        oob_mask = ~np.isin(np.arange(len(X)), bootstrap_indices)
        
        X_boot, y_boot = X[bootstrap_indices], y[bootstrap_indices]
        X_oob, y_oob = X[oob_mask], y[oob_mask]

        # Train the model on in-bag data
        model.fit(X_boot, y_boot)
        y_pred_train = model.predict(X_boot)

        # Collect train metrics
        train_metrics.append({
            'accuracy': accuracy_score(y_boot, y_pred_train),
            'precision': precision_score(y_boot, y_pred_train, average='weighted'),
            'recall': recall_score(y_boot, y_pred_train, average='weighted'),
            'f1': f1_score(y_boot, y_pred_train, average='weighted'),
        })

        # Collect test metrics if there are out-of-bag samples
        if len(y_oob) > 0:
            y_pred_oob = model.predict(X_oob)
            test_metrics.append({
                'accuracy': accuracy_score(y_oob, y_pred_oob),
                'precision': precision_score(y_oob, y_pred_oob, average='weighted'),
                'recall': recall_score(y_oob, y_pred_oob, average='weighted'),
                'f1': f1_score(y_oob, y_pred_oob, average='weighted'),
            })
            confusion_matrices.append(confusion_matrix(y_oob, y_pred_oob))

    # Aggregate train and test metrics
    train_metrics_mean = {metric: np.mean([m[metric] for m in train_metrics]) for metric in train_metrics[0]}
    test_metrics_mean = {metric: np.mean([m[metric] for m in test_metrics]) for metric in test_metrics[0]}

    # Combine metrics using the .632 formula
    combined_metrics = {metric: 0.368 * train_metrics_mean[metric] + 0.632 * test_metrics_mean[metric] for metric in train_metrics_mean}
    mean_confusion_matrix = np.mean(confusion_matrices, axis=0) if confusion_matrices else None

    combined_metrics['confusion_matrix'] = mean_confusion_matrix
    return combined_metrics

# Evaluate models
metrics_results = {}
confusion_matrices = {}
crossval_results = {}
bootstrap_results = {}


for model_name, model in models.items():
    # Cross-validation
    crossval_metrics = cross_validate_method(model, X, y)
    
    # Bootstrap
    bootstrap_metrics = bootstrap_632_method(model, X, y)
    
    # Save metrics
    metrics_results[model_name] = {
        'CrossVal Accuracy': crossval_metrics['accuracy'],
        'CrossVal Precision': crossval_metrics['precision'],
        'CrossVal Recall': crossval_metrics['recall'],
        'CrossVal F1-Score': crossval_metrics['f1'],
        'Bootstrap Accuracy': bootstrap_metrics['accuracy'],
        'Bootstrap Precision': bootstrap_metrics['precision'],
        'Bootstrap Recall': bootstrap_metrics['recall'],
        'Bootstrap F1-Score': bootstrap_metrics['f1']
    }
    
    # Save confusion matrices
    crossval_results[model_name] = crossval_metrics['confusion_matrix']
    bootstrap_results[model_name] = bootstrap_metrics['confusion_matrix']

# 3. Build Dash App
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Titanic Classification Models", style={'textAlign': 'center'}),

    # Performance Metrics
    html.H3("Performance Metrics"),
    dcc.Graph(figure=px.bar(
        pd.DataFrame(metrics_results).reset_index(),
        x='index', y=list(models.keys()),
        barmode='group',
        title="Performance Metrics for All Models",
        labels={'index': 'Metrics', 'value': 'Score'}
    )),

    html.H3("Confusion Matrices"),
    html.Div([
        dcc.Graph(figure=px.imshow(confusion_matrices[model], text_auto=True, title=f"{model}", width=300, height=300))
        for model in models.keys()
    ], style={'display': 'flex', 'flexDirection': 'row'}),

    # Cross-validation
    html.H3("Cross-Validation Results"),
    dcc.Graph(figure=px.line(
        x=list(range(1, 11)),
        y=[crossval_results[model] for model in models.keys()],
        title="Cross-Validation Accuracy (10-Fold)",
        labels={'x': 'Fold', 'y': 'Accuracy', 'color': 'Models'}
    )),

    # Bootstrap .632 Results
    html.H3("Bootstrap .632 Results"),
    dcc.Graph(figure=px.bar(
        x=list(models.keys()),
        y=[bootstrap_results[model] for model in models.keys()],
        title="Bootstrap .632 Accuracy",
        labels={'x': 'Model', 'y': 'Bootstrap Accuracy', 'color': 'Models'},
        color=list(models.keys())
    )),
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [37]:
# Import libraries
import base64
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, dcc, html

# 1. DATA PREPARATION
df = pd.read_csv('titanic.csv')

# Data Cleaning
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], errors='ignore')
df['Age'] = np.floor(df['Age'])
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
df = pd.get_dummies(df, columns=['Sex', 'Embarked'], drop_first=True)

# Features and target
X = df.drop(columns=['Survived'])
y = df['Survived']

# Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "KNN (k=3)": KNeighborsClassifier(n_neighbors=3)
}

# Cross-validation and Bootstrap methods
def cross_validate_method(model, X, y):
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    return cross_val_score(model, X, y, cv=cv, scoring='accuracy')

def bootstrap_632_method(model, X, y):
    train_acc, test_acc = [], []
    for _ in range(100):
        X_boot, y_boot = resample(X, y, replace=True)
        X_oob = X[~np.in1d(np.arange(X.shape[0]), X_boot.index)]
        y_oob = y[~np.in1d(np.arange(y.shape[0]), X_boot.index)]
        model.fit(X_boot, y_boot)
        acc_train = accuracy_score(y_boot, model.predict(X_boot))
        acc_test = accuracy_score(y_oob, model.predict(X_oob)) if len(y_oob) > 0 else 0
        train_acc.append(acc_train)
        test_acc.append(acc_test)
    return np.mean(0.368 * np.array(train_acc) + 0.632 * np.array(test_acc))

# Evaluate models
metrics_results = {}
confusion_matrices = {}
crossval_results = {}
bootstrap_results = {}

for model_name, model in models.items():
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Save metrics
    metrics_results[model_name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred)
    }
    confusion_matrices[model_name] = confusion_matrix(y_test, y_pred)
    crossval_results[model_name] = cross_validate_method(model, X, y)
    bootstrap_results[model_name] = bootstrap_632_method(model, X, y)

# 3. Build Dash App
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Titanic Classification Models", style={'textAlign': 'center'}),

    # Performance Metrics
    html.H3("Performance Metrics"),
    dcc.Graph(figure=px.bar(
        pd.DataFrame(metrics_results).reset_index(),
        x='index', y=list(models.keys()),
        barmode='group',
        title="Performance Metrics for All Models",
        labels={'index': 'Metrics', 'value': 'Score'}
    )),

    # Confusion Matrices
    html.H3("Confusion Matrices"),
    html.Div([
        dcc.Graph(figure=px.imshow(confusion_matrices[model], text_auto=True, title=f"Confusion Matrix: {model}", width=300, height=300))
        for model in models.keys()
    ], style={'display': 'flex', 'flexDirection': 'row', 'flexWrap': 'wrap'}),
    
    # Cross-validation
    html.H3("Cross-Validation Results"),
    dcc.Graph(figure=px.line(
        x=list(range(1, 11)),
        y=[crossval_results[model] for model in models.keys()],
        title="Cross-Validation Accuracy (10-Fold)",
        labels={'x': 'Fold', 'y': 'Accuracy', 'color': 'Models'}
    )),

    # Bootstrap .632 Results
    html.H3("Bootstrap .632 Results"),
    dcc.Graph(figure=px.bar(
        x=list(models.keys()),
        y=[bootstrap_results[model] for model in models.keys()],
        title="Bootstrap .632 Accuracy",
        labels={'x': 'Model', 'y': 'Bootstrap Accuracy'},
        color=list(models.keys()),
        # color_discrete_sequence=px.colors.qualitative.Set1
    )),


])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

/var/folders/y1/1tjj64zd37sdph6807cm97nh0000gn/T/ipykernel_31670/371262913.py:24: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



/var/folders/y1/1tjj64zd37sdph6807cm97nh0000gn/T/ipykernel_31670/371262913.py:25: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing '